# Spectral Mesh Flattening

by Ana + Gabrielle

In [100]:
import trimesh
import numpy as np
import scipy
from PIL import Image

In [101]:
def boundary(mesh):
    
    """ A function to find the boundary edges and vertices of a mesh. 
        Inputs:
        mesh: a trimesh mesh. 
        Outputs: 
        boundary_vertices: a list of the indices of vertices on the boundary. 
        next_vertex: a dictionary to find the next vertex on the boundary. 
    """

    next_vertex = {}
    boundary_vertices = []

    edges = mesh.edges
    vertex_faces = mesh.vertex_faces
    
    for i in range(len(edges)):

        edge = edges[i]
        v1, v2 = edge
        faces = [j for j in vertex_faces[v1] if j != -1 and j in vertex_faces[v2]]
        
        # Boundary edges are edges that are only in one face
        if len(faces) == 1:

            next_vertex[v1] = v2
            boundary_vertices = boundary_vertices + [v1, v2]
    
    # Remove any duplicates
    boundary_vertices = np.unique(np.array(boundary_vertices))
    
    return boundary_vertices, next_vertex

In [102]:
## This is to export the boundaries coloured. 
# face = trimesh.load("Face.obj")
# b_verts, next_vertex = boundary(face)
# face.visual.vertex_colors[b_verts] = [255, 0, 0, 1]
# face.visual.vertex_colors[1] = [0, 255, 0, 1]
# face.visual.vertex_colors[3] = [0, 0, 255, 1]
# face.export("boundaries.obj")

In [111]:
def circle_boundary(mesh, centre = [0, 0]):

    """ A function to map the boundary of a mesh to a circle centered at the origin. 
        Inputs:
        mesh: a trimesh mesh. 
        centre: optional argument to change the centre of the circle. 
        
        Outputs: 
        new_boundary_values: an array of boundary values (2 dimensional) on the circle. 
    """

    vertices = mesh.vertices

    range_x = (np.max(vertices[:, 0]) - np.min(vertices[:, 0]))/2
    range_y = (np.max(vertices[:, 1]) - np.min(vertices[:, 1]))/2
    range_z = (np.max(vertices[:, 2]) - np.min(vertices[:, 2]))/2

    # Keep the circle roughly the same size as the mesh by choosing half the maximum range
    # as the radius 
    radius = np.max([range_x, range_y, range_z])

    # find the boundary information, and intialize the output 
    b_verts, next_vertex = boundary(mesh)
    output = np.zeros((len(vertices), 2))
    weights = {}
    
    # Calculate the "weights" of each edge and store in a dictionary 
    # to be used to determine how far from the other vertices each edge 
    for i in range(len(b_verts)):
        weights[b_verts[i]] = np.linalg.norm(vertices[b_verts[i]] - vertices[next_vertex[b_verts[i]]])
    
    total = sum(weights.values(), 0.0)
    weights = {k: v / total for k, v in weights.items()}

    # Start with one point set at 0 degrees, and then increase the angle each time (since arc length is
    # proportional to angle)
    angle_sum = 0
    v1 = b_verts[0]
    output[v1] = [centre[0] + radius * np.cos(angle_sum), centre[1] + radius * np.sin(angle_sum)]

    for i in range(len(b_verts)):
        
        angle = weights[v1] * 2 * np.pi
        angle_sum += angle

        # calculate the new positions
        new_x = centre[0] + radius * np.cos(angle_sum)
        new_y = centre[1] + radius * np.sin(angle_sum)

        v1 = next_vertex[v1]
        output[v1] = [new_x, new_y]

    # make sure the outputs are in the same order as the b_verts 
    new_boundary_values = output[b_verts]

    return new_boundary_values

def get_quartile_length(mesh_vertices, next_vertex_dict):
    """ A function to find 1/4 of the length of all distances around the boundary of a mesh. 
        Inputs:
        mesh_vertices: array of all vertices of a trimesh.
        next_vertex_dict: a dictionary to find the next vertex on the boundary.  

        Outputs:
        quartile_length: 1/4 of the length of total distance around boundary.
        distances: a dictionary to find the distance of a vertex from its previous vertex.
    """
        
    # get first vertex index on boundary
    current_vertex_idx = list(next_vertex_dict.keys())[0]

    # keep track of distances
    distances = {}
    total_dist = 0.0
    for i in range(len(next_vertex_dict)):
        # get currrent vertex values
        current_vertex = mesh_vertices[current_vertex_idx]

        # get next vertex index
        next_vertex_idx = next_vertex_dict[current_vertex_idx]
        # get next vertex value
        next_vertex = mesh_vertices[next_vertex_dict[current_vertex_idx]]

        dist = np.linalg.norm(next_vertex - current_vertex)
        distances[next_vertex_idx] = dist
        total_dist += dist

        # reset current vertex idk
        current_vertex_idx = next_vertex_idx

    quartile_length = total_dist/4


    return quartile_length, distances

def square_boundary(mesh):
    """ A function to map the boundary of a mesh to a square. 
        Inputs:
        mesh: a trimesh mesh. 
        quartile_dist: float value for the 1/4 of the total distance around boundary
        dist_dict: dictionary that maps vertex to a distance from its previous vertex
        
        Outputs: 
        new_boundary_values: an array of boundary values (2 dimensional) on the circle. 
    """

    mesh_vertices = mesh.vertices
    output = np.zeros((len(mesh_vertices), 2))

    range_x = (np.max(mesh_vertices[:, 0]) - np.min(mesh_vertices[:, 0]))/2
    range_y = (np.max(mesh_vertices[:, 1]) - np.min(mesh_vertices[:, 1]))/2
    range_z = (np.max(mesh_vertices[:, 2]) - np.min(mesh_vertices[:, 2]))/2

    # keep the sqaure roughly the same size as the mesh by choosing half of maximum range as unit
    unit = np.max([range_x, range_y, range_z])

    unit_square = np.array([[0.0,0.0], 
                        [unit,0.0], 
                        [unit,unit],  
                        [0.0,unit]])
    
    boundary_vertices, next_vertex_dict = boundary(mesh)
    quartile_dist, dist_dict = get_quartile_length(mesh_vertices, next_vertex_dict)

    # keep track of total_dist for each edge of square 
    total_dist = 0.0
    # get first vertex index on boundary
    starting_vertex_idx = list(next_vertex_dict.keys())[0]
    current_vertex_idx = starting_vertex_idx
    next_vertex_idx = next_vertex_dict[current_vertex_idx]
    # track vertices to remap (include first two vertices in map)
    vertices_to_map = [current_vertex_idx, next_vertex_idx]
    # include their respective distances to current_vertex
    dist = dist_dict[next_vertex_idx]
    dist_list = [0.0, dist]
    for i in range(4): #loop for each side of square
        # track corners
        A = unit_square[i % len(unit_square)]
        B = unit_square[(i + 1) % len(unit_square)]

        # sum up distances on edge until we reach 1/4 of total distace
        # if final edge of square then sum up all remaining distances (as division will never be perfect)
        while(total_dist + dist < quartile_dist or i == 3):
            next_vertex_idx = next_vertex_dict[current_vertex_idx]

            total_dist += dist
            dist_list.append(total_dist)
            vertices_to_map.append(next_vertex_idx)
            current_vertex_idx = next_vertex_idx
            dist = dist_dict[current_vertex_idx]
            # break once back at starting vertex 
            if(current_vertex_idx == starting_vertex_idx):
                break
        # calculate new positions in two dimensions
        output[vertices_to_map] = [d/total_dist * (B - A) + A for d in dist_list]
        
        # update values for next edge of sqaure
        total_dist = dist
        dist_list = [0.0, dist]
        next_vertex_idx = next_vertex_dict[current_vertex_idx]
        vertices_to_map = [current_vertex_idx, next_vertex_idx]
        current_vertex_idx = next_vertex_idx

    new_boundary_values = output[boundary_vertices]
    return new_boundary_values

In [104]:
# This cell exports the mesh with the boundary vertices transformed to the 2d circle. 
# import matplotlib.pyplot as plt
# %matplotlib inline

# face = trimesh.load("Horse.obj")
# xy = np.array(circle_boundary(face))
# x = xy[:, 0]
# y = xy[:, 1]
# plt.scatter(x, y, marker = 'x')
# b_verts, _ = boundary(face)
# face.vertices[b_verts] = np.hstack((xy, -12*np.ones((len(x), 1))))
# face.export("CircleExperiment.obj")

In [112]:
def angle(v0, v1, v2):

    """ A function to work out the angle between two edges. 
        Inputs:
        v0: vertex coordinates at the centre of the edges
        v1: vertex of edge1
        v2: vertex of edge2
        
        returns: angle between edge from v0 to v1 and edge from v0 to v2
    """

    edge1 = v1 - v0
    edge2 = v2 - v0

    return np.arccos(np.clip(np.dot(edge1, edge2)/(np.linalg.norm(edge1)*np.linalg.norm(edge2)), -1, 1))

In [113]:
def uniformLaplaceBeltrami(mesh):

    """ A function that creates the uniform Laplacian matrix for a given mesh. 
        Inputs:
        mesh: a trimesh mesh
        
        Outputs:
        L: sparse csc matrix containing -1 on the diagonal and 1/k on the off diagonal neighbours.
    """

    n = len(mesh.vertices)
    L = scipy.sparse.lil_matrix(-1*np.eye(n))
    neighbours = mesh.vertex_neighbors

    for i in range(n):
        L[i, neighbours[i]] = 1/len(neighbours[i])

    return L.tocsr()

def cotanLaplaceBeltrami(mesh):

    """ A function to find the cotangent discretization of the Laplacian. 
        Inputs:
        mesh: a trimesh mesh
        Outputs:
        L: the cotangent discretization of the Laplacian. 
    """

    vs = mesh.vertices
    fs = mesh.faces
    vert_ns = mesh.vertex_neighbors
    vert_fs = mesh.vertex_faces

    n = len(vs)
    areas = np.zeros(n)
    C = scipy.sparse.lil_matrix(np.zeros((n, n)))
    
    for vertex in range(n):

        # for every vertex find the neighbours and the faces
        faces = vert_fs[vertex][vert_fs[vertex] != -1]

        for neighbour in vert_ns[vertex]:
            # for every neighbour find the faces that are shared with the vertex, and the other vertices on those faces
            faces = [i for i in vert_fs[vertex] if neighbour in fs[i] and i != -1]
            vertices = [i for i in fs[faces].flatten() if not i in [vertex, neighbour, -1]]

            if len(vertices) > 1:
                
                angle1 = angle(vs[vertices[0]], vs[vertex], vs[neighbour])
                angle2 = angle(vs[vertices[1]], vs[vertex], vs[neighbour])
            
            # This is the case where the mesh is not closed
            else:
                angle1 = angle(vs[vertices[0]], vs[vertex], vs[neighbour])
                angle2 = angle1

            # account for division by zero (and near zero)
            if abs(np.tan(angle1)) < 1e-10:
                cot_alpha = 1e10
            else:
                cot_alpha = 1/np.tan(angle1)

            if abs(np.tan(angle2)) < 1e-10:
                cot_beta = 1e10
            else:
                cot_beta = 1/np.tan(angle2)
            
            # Assign the values 
            C[vertex, neighbour] = cot_alpha + cot_beta
            C[vertex, vertex] -= cot_alpha + cot_beta 

            # voronoi areas 
            areas[vertex] += (C[vertex, neighbour] * np.linalg.norm(vs[vertex] - vs[neighbour])**2) / 8

    M_inv = scipy.sparse.diags(1/(2*areas))
    
    L = (M_inv @ C)
    return L.tocsr()

def mvLaplaceBeltrami(mesh):

    """ A function to find the mean value weighted discretization of the Laplace Beltrami.
        Inputs:
        mesh: a trimesh mesh
        Outputs:
        L: the mean value discretization of the Laplacian.
    """

    vs = mesh.vertices
    fs = mesh.faces
    vert_ns = mesh.vertex_neighbors
    vert_fs = mesh.vertex_faces

    n = len(vs)
    L = scipy.sparse.lil_matrix(np.zeros((n, n)))

    for vertex in range(n):
        # for every vertex find the neighbours and the faces 
        faces = vert_fs[vertex][vert_fs[vertex] != -1]

        for neighbour in vert_ns[vertex]:
            # for every neighbour find the faces that are shared with the vertex, and the other vertices on those faces
            faces = [i for i in vert_fs[vertex] if neighbour in fs[i] and i != -1] 
            vertices = [i for i in fs[faces].flatten() if not i in [vertex, neighbour, -1]]

            if len(vertices) > 1:

                angle1 = angle(vs[vertex], vs[vertices[0]], vs[neighbour])/2
                angle2 = angle(vs[vertex], vs[vertices[1]], vs[neighbour])/2
            
            # This is the case where the mesh is not closed
            else:
                angle1 = angle(vs[vertex], vs[vertices[0]], vs[neighbour])/2
                angle2 = angle1/2
            
            # Assign the values 
            L[vertex, neighbour] = (np.tan(angle1) + np.tan(angle2))/np.linalg.norm(vs[vertex] - vs[neighbour])
            L[vertex, vertex] -= (np.tan(angle1) + np.tan(angle2))/np.linalg.norm(vs[vertex] - vs[neighbour])

    return L.tocsr()    

In [114]:
def tutte_embedding(mesh, boundary_function = circle_boundary, LaplaceBeltrami = cotanLaplaceBeltrami):

    """ A function that uses Tutte's embedding method to flatten a mesh
        with given boundary conditions. 
        Inputs: 
        mesh: a trimesh mesh. 
        boundary_function: a function that returns the 2D coordinates for the boundary vertices.
         
        Outputs: 
        flat_mesh: the flattened mesh. 
    """

    flat_mesh = mesh.copy()

    vertices = mesh.vertices
    b_verts, _ = boundary(mesh)
    b_vals = boundary_function(mesh)
    L = LaplaceBeltrami(mesh)
    L = L.tolil()
    
    for vert in b_verts:
        L[vert, :] = np.zeros(L.shape[1])
        L[vert, vert] = 1
    
    L = L.tocsc()
    
    B = np.zeros((len(vertices), 2))
    B[b_verts] = b_vals

    X = scipy.sparse.linalg.spsolve(L, B)
    flat_mesh.vertices = np.hstack((X, np.zeros((X.shape[0], 1))))

    return flat_mesh

In [124]:
def export_textured(mesh_filename, export_name, flat_export_name, bound_func=circle_boundary, texture_file='textures/CheckTexture.png', LB=cotanLaplaceBeltrami):

    """ A function to export a mesh and a flattened mesh with a texture using the 
        flat mesh as the UV parametrization. 
        
        Inputs:
        mesh_filename: mesh to import. 
        export_name: name that the original mesh (with texture) will be saved to. 
        flat_export_name: flattened textured mesh export name. 
        texture_name: optional argument to determine which texture to apply. 
    """

    mesh = trimesh.load(mesh_filename)
    flat_mesh = tutte_embedding(mesh, boundary_function=bound_func, LaplaceBeltrami=LB)

    uv_coordinates = flat_mesh.vertices[:, :2]
    texture = Image.open(texture_file)

    mesh.visual = trimesh.visual.TextureVisuals(uv=uv_coordinates)
    flat_mesh.visual = trimesh.visual.TextureVisuals(uv=uv_coordinates)

    mesh.visual.material.image = texture
    flat_mesh.visual.material.image = texture

    mesh.export(export_name, include_texture=True)
    flat_mesh.export(flat_export_name, include_texture=True)

    return 

In [126]:
export_textured("meshes/Rhino.obj", "c_meshes/uniformRhino.obj", "c_meshes/texFlatRhino1.obj", bound_func=square_boundary, LB = uniformLaplaceBeltrami)
export_textured("meshes/Rhino.obj", "c_meshes/cotanRhino.obj", "c_meshes/texFlatRhino2.obj", bound_func=square_boundary, LB = cotanLaplaceBeltrami)
export_textured("meshes/Rhino.obj", "c_meshes/mvRhino.obj", "c_meshes/texFlatRhino3.obj", bound_func=square_boundary, LB = mvLaplaceBeltrami)